En el siguiente notebook, a partir de los datos escrapeados en el notebook 𝑋𝑋𝑋𝑋, se realizará una limpieza inicial de las distintas tablas de datos prévia a la transformación y creación de las distintas variables que van a ser usadas para el entrenamiento del modelo.

Primero, importamos todos aquellos paquetes que vamos a usar a lo largo del notebook.

In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup, SoupStrainer
import sys
import string
import missingno as msno 
import seaborn as sn
import requests
import datetime
import progressbar
import time
import re
from selenium import webdriver
#from matplotlib import pyplot
from webdriver_manager.chrome import ChromeDriverManager
from pandas_profiling import ProfileReport
from pandas.plotting import scatter_matrix
from matplotlib import pyplot

import warnings
warnings.filterwarnings("ignore")

from sklearn.impute import KNNImputer


# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.rc('font', size=12) 
plt.rc('figure', figsize = (12, 5))

# Settings for the visualizations
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 2,'font.family': [u'times']})

# 2. Data cleaning

### 2.1. Importamos los archivos raw

In [3]:
all_players_raw = pd.read_excel('Scraped data/scraped_data_v2/combined_drafted_and_all_players.xlsx',index_col=0)
col_conf_adv_raw = pd.read_csv('Scraped data/scraped_data_v2/college_stats_CONF_ADV_BS_df.csv',index_col=0)
col_conf_pg_raw = pd.read_csv('Scraped data/scraped_data_v2/college_stats_CONF_PG_BS_df.csv',index_col=0)
col_conf_tot_raw = pd.read_csv('Scraped data/scraped_data_v2/college_stats_CONF_TOT_BS_df.csv',index_col=0)
col_rs_adv_raw = pd.read_csv('Scraped data/scraped_data_v2/college_stats_RS_ADV_BS_df.csv',index_col=0)
col_rs_pg_raw = pd.read_csv('Scraped data/scraped_data_v2/college_stats_RS_PG_BS_df.csv',index_col=0)
col_rs_tot_raw = pd.read_csv('Scraped data/scraped_data_v2/college_stats_RS_TOT_BS_df.csv',index_col=0)
col_extra_info_raw = pd.read_csv('Scraped data/scraped_data_v2/college_player_extra_info_df.csv',index_col=0)

In [4]:
col_conf_pg_raw['SEASON'].unique()

array(['2012-13', '2016-17', '2017-18', '2015-16', '2018-19', '2019-20',
       '2010-11', '2011-12', '2020-21', '2014-15', '2013-14'],
      dtype=object)

In [5]:
col_conf_pg_raw[col_conf_adv_raw['COLLEGE_URL']=='/players/corey-brewer-1.html'] #2004-07 https://www.sports-reference.com/cbb/players/corey-brewer-1.html

,SEASON,SCHOOL,CONF,G,GS,MP,FG,FGA,FG%,2P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,COLLEGE_URL,3P%


In [6]:
col_rs_adv_raw[col_rs_adv_raw['COLLEGE_URL']=='/players/corey-brewer-1.html']

,SEASON,SCHOOL,CONF,G,MP,TS%,eFG%,3PAr,FTr,TOV%,...,ORB%,DRB%,TRB%,AST%,STL%,BLK%,USG%,OBPM,DBPM,BPM
191,2004-05,Florida,SEC,32.0,791.0,0.574,0.565,0.231,0.509,19.9,...,NaN,NaN,8.2,14.6,NaN,1.2,16.8,NaN,NaN,NaN
192,2005-06,Florida,SEC,39.0,1095.0,0.571,0.526,0.329,0.392,20.0,...,NaN,NaN,10.0,22.4,NaN,1.0,25.0,NaN,NaN,NaN
193,2006-07,Florida,SEC,37.0,1053.0,0.570,0.531,0.334,0.435,17.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,2004-05,Florida,SEC,32.0,791.0,0.574,0.565,0.231,0.509,19.9,...,NaN,NaN,8.2,14.6,NaN,1.2,16.8,NaN,NaN,NaN
195,2005-06,Florida,SEC,39.0,1095.0,0.571,0.526,0.329,0.392,20.0,...,NaN,NaN,10.0,22.4,NaN,1.0,25.0,NaN,NaN,NaN
196,2006-07,Florida,SEC,37.0,1053.0,0.570,0.531,0.334,0.435,17.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
col_conf_tot_raw[col_conf_adv_raw['COLLEGE_URL']=='/players/frank-mason-1.html'] #1997-2011 https://www.sports-reference.com/cbb/players/shane-battier-1.html

,SEASON,SCHOOL,CONF,G,GS,MP,FG,FGA,FG%,2P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,COLLEGE_URL,3P%


In [8]:
col_rs_tot_raw[col_rs_tot_raw['COLLEGE_URL']=='/players/frank-mason-1.html']

,SEASON,SCHOOL,CONF,G,MP,FG,FGA,FG%,2P,2PA,...,AST,STL,BLK,TOV,PF,PTS,COLLEGE_URL,GS,ORB,DRB


### 2.2. all_players_raw

In [9]:
all_players_raw = all_players_raw[(all_players_raw['ACTIVE_FROM']>1978)]

Exploramos el primer csv en busca de variables de tipo incorrecto, datos vacíos y similares

In [10]:
all_players_raw.shape

(3339, 12)

In [11]:
all_players_raw.head()

,URL,NAME,ACTIVE_FROM,ACTIVE_TO,POSITION,COLLEGE,HEIGHT,WEIGHT,BIRTH_DATE,DRAFT_YEAR,DRAFT_PICK,DRAFT_TEAM
0,/players/a/abdelal01.html,Alaa Abdelnaby,1991,1995,F-C,Duke,6-10,240.0,1968-06-24,1990.0,25.0,POR
3,/players/a/abdulma02.html,Mahmoud Abdul-Rauf,1991,2001,G,LSU,6-1,162.0,1969-03-09,1990.0,3.0,DEN
4,/players/a/abdulta01.html,Tariq Abdul-Wahad,1998,2003,F,"Michigan, San Jose State",6-6,223.0,1974-11-03,1997.0,11.0,SAC
5,/players/a/abdursh01.html,Shareef Abdur-Rahim,1997,2008,F,California,6-9,225.0,1976-12-11,1996.0,3.0,VAN
9,/players/a/abrinal01.html,Álex Abrines,2017,2019,G-F,NaN,6-6,200.0,1993-08-01,NaN,NaN,NaN


#### 2.2.1. Revisar los tipos de datos

In [12]:
all_players_raw.dtypes #Todos los tipos son correctos, a excepción del HEIGHT, que no lo usaremos por no ser en cm

URL                    object
NAME                   object
ACTIVE_FROM             int64
ACTIVE_TO               int64
POSITION               object
COLLEGE                object
HEIGHT                 object
WEIGHT                float64
BIRTH_DATE     datetime64[ns]
DRAFT_YEAR            float64
DRAFT_PICK            float64
DRAFT_TEAM             object
dtype: object

#### 2.2.2. Buscar y rellenar los datos vacíos

Estos son los datos vacíos:

In [13]:
all_players_raw.isnull().sum()

URL              0
NAME             0
ACTIVE_FROM      0
ACTIVE_TO        0
POSITION         0
COLLEGE        324
HEIGHT           0
WEIGHT           0
BIRTH_DATE       0
DRAFT_YEAR     962
DRAFT_PICK     963
DRAFT_TEAM     962
dtype: int64

Los COLLEGE vacíos se deben a jugadores que no han ido al COLLEGE. Han ido directamente a la NBA sin pasar por el college (Kevin Garnett) o han llegado desde Europa (Álex Abrines). Los rellenamos con 'No College'

In [14]:
all_players_raw[(all_players_raw['NAME']=='Kevin Garnett') | (all_players_raw['NAME'] == 'Álex Abrines')]

,URL,NAME,ACTIVE_FROM,ACTIVE_TO,POSITION,COLLEGE,HEIGHT,WEIGHT,BIRTH_DATE,DRAFT_YEAR,DRAFT_PICK,DRAFT_TEAM
9,/players/a/abrinal01.html,Álex Abrines,2017,2019,G-F,NaN,6-6,200.0,1993-08-01,NaN,NaN,NaN
1565,/players/g/garneke01.html,Kevin Garnett,1996,2016,F-C,NaN,6-11,240.0,1976-05-19,1995.0,5.0,MIN


In [15]:
all_players = all_players_raw.copy()
all_players['COLLEGE'] = all_players['COLLEGE'].fillna('No College')

Los que tienen 'DRAFT_YEAR', 'DRAFT_PICK' y 'DRAFT_TEAM' no han sido seleccionados en el draft, por lo que no interesan

In [16]:
all_players[all_players['DRAFT_PICK'].isna() & all_players['DRAFT_YEAR']>0] #Undrafted - https://en.wikipedia.org/wiki/Bill_Edwards_(basketball)

,URL,NAME,ACTIVE_FROM,ACTIVE_TO,POSITION,COLLEGE,HEIGHT,WEIGHT,BIRTH_DATE,DRAFT_YEAR,DRAFT_PICK,DRAFT_TEAM
1283,/players/e/edwarbi01.html,Bill Edwards,1994,1994,F,Wright State University,6-8,215.0,1971-09-22,1953.0,NaN,ROC


In [17]:
all_players_1 = all_players.dropna() #Eliminamos los undrafted

In [18]:
all_players_1.reset_index()

,index,URL,NAME,ACTIVE_FROM,ACTIVE_TO,POSITION,COLLEGE,HEIGHT,WEIGHT,BIRTH_DATE,DRAFT_YEAR,DRAFT_PICK,DRAFT_TEAM
0,0,/players/a/abdelal01.html,Alaa Abdelnaby,1991,1995,F-C,Duke,6-10,240.0,1968-06-24,1990.0,25.0,POR
1,3,/players/a/abdulma02.html,Mahmoud Abdul-Rauf,1991,2001,G,LSU,6-1,162.0,1969-03-09,1990.0,3.0,DEN
2,4,/players/a/abdulta01.html,Tariq Abdul-Wahad,1998,2003,F,"Michigan, San Jose State",6-6,223.0,1974-11-03,1997.0,11.0,SAC
3,5,/players/a/abdursh01.html,Shareef Abdur-Rahim,1997,2008,F,California,6-9,225.0,1976-12-11,1996.0,3.0,VAN
4,10,/players/a/achiupr01.html,Precious Achiuwa,2021,2022,F,Memphis,6-8,225.0,1999-09-19,2020.0,20.0,MIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2371,5240,/players/z/zhizhwa01.html,Wang Zhizhi,2001,2005,C,No College,7-0,255.0,1977-07-08,1999.0,36.0,DAL
2372,5242,/players/z/zimmede01.html,Derrick Zimmerman,2006,2006,G,Mississippi State,6-3,195.0,1981-12-02,2003.0,40.0,GSW
2373,5243,/players/z/zimmest01.html,Stephen Zimmerman,2017,2017,C,UNLV,7-0,240.0,1996-09-09,2016.0,41.0,ORL
2374,5244,/players/z/zipsepa01.html,Paul Zipser,2017,2018,G-F,No College,6-8,215.0,1994-02-18,2016.0,48.0,CHI


#### 2.2.3. Modificaciones previas necesarias

Modificamos 'BIRTH_DATE', 'HEIGHT' y 'WEIGHT' para que tengan unidades de medida más manejables

In [19]:
all_players_1['BIRTH_DATE(year)'] = all_players_1['BIRTH_DATE'].dt.strftime('%Y')
all_players_1['BIRTH_DATE(year)'] = all_players_1['BIRTH_DATE(year)'].apply(lambda x: int(x))
all_players_1['HEIGHT(cm)'] = all_players_1['HEIGHT'].apply(lambda x: float(x[:1])*30.48 + float(x[2:])*2.54 )
all_players_1['WEIGHT(kg)'] = all_players_1['WEIGHT'].apply(lambda x: round(x/2.204622,2))

In [20]:
all_players_2 = all_players_1.drop(['BIRTH_DATE', 'HEIGHT', 'WEIGHT'], 1)

In [21]:
all_players_2.isnull().sum() #OK

URL                 0
NAME                0
ACTIVE_FROM         0
ACTIVE_TO           0
POSITION            0
COLLEGE             0
DRAFT_YEAR          0
DRAFT_PICK          0
DRAFT_TEAM          0
BIRTH_DATE(year)    0
HEIGHT(cm)          0
WEIGHT(kg)          0
dtype: int64

In [22]:
all_players_2.dtypes #OK

URL                  object
NAME                 object
ACTIVE_FROM           int64
ACTIVE_TO             int64
POSITION             object
COLLEGE              object
DRAFT_YEAR          float64
DRAFT_PICK          float64
DRAFT_TEAM           object
BIRTH_DATE(year)      int64
HEIGHT(cm)          float64
WEIGHT(kg)          float64
dtype: object

In [23]:
all_players_2.count()

URL                 2376
NAME                2376
ACTIVE_FROM         2376
ACTIVE_TO           2376
POSITION            2376
COLLEGE             2376
DRAFT_YEAR          2376
DRAFT_PICK          2376
DRAFT_TEAM          2376
BIRTH_DATE(year)    2376
HEIGHT(cm)          2376
WEIGHT(kg)          2376
dtype: int64

Guardamos una copia limpia del dataset

In [25]:
df_all_players_clean = all_players_2.copy()
df_all_players_clean.to_excel('Clean_data/Data_Clean_V1/All_players_clean.xlsx')

### 2.3. col_conf_adv_raw

In [27]:
col_conf_adv_raw['year'] = col_conf_adv_raw['SEASON'].apply(lambda x: int(x[:4]))
col_conf_adv_raw = col_conf_adv_raw[col_conf_adv_raw['year']>1985]
col_conf_adv_raw = col_conf_adv_raw.drop(['year'],1)

In [28]:
col_conf_adv_raw.head()

,SEASON,SCHOOL,CONF,G,GS,MP,PER,TS%,eFG%,3PAr,...,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/40,COLLEGE_URL
0,2012-13,Pitt,Big East,17,17,413,20.7,0.536,0.543,0.000,...,5.3,1.5,11.2,14.8,17.0,0.7,0.9,1.6,0.157,/players/steven-adams-1.html
1,2016-17,Kentucky,SEC,18,18,557,23.3,0.680,0.667,0.008,...,2.6,1.4,5.2,14.4,18.3,1.8,1.1,2.8,0.203,/players/edrice-adebayo-1.html
2,2016-17,Texas,Big 12,18,18,622,23.2,0.602,0.599,0.015,...,4.4,1.1,5.3,17.4,24.0,1.2,0.8,2.0,0.127,/players/jarrett-allen-1.html
3,2017-18,Dayton,A-10,16,0,213,16.4,0.612,0.620,0.109,...,5.5,0.9,5.8,21.0,22.0,0.3,0.1,0.4,0.082,/players/kostas-antetokounmpo-1.html
4,2015-16,Indiana,Big Ten,18,0,281,22.5,0.591,0.593,0.293,...,7.2,3.6,5.0,11.0,18.1,0.7,0.6,1.3,0.186,/players/og-anunoby-1.html


In [29]:
col_conf_adv_raw.isnull().sum()

SEASON         0
SCHOOL         0
CONF           0
G              0
GS             0
MP             0
PER            0
TS%            2
eFG%           2
3PAr           2
FTr            2
PProd          0
ORB%           0
DRB%           0
TRB%           0
AST%           0
STL%           0
BLK%           0
TOV%           2
USG%           0
OWS            0
DWS            0
WS             0
WS/40          0
COLLEGE_URL    0
dtype: int64

In [30]:
col_conf_adv_raw[col_conf_adv_raw.isna().any(axis=1)] #Estos jugadores casi no jugaron en la temporada 2010/11. Lo llenamos con 0.

,SEASON,SCHOOL,CONF,G,GS,MP,PER,TS%,eFG%,3PAr,...,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/40,COLLEGE_URL
171,2010-11,Weber State,Big Sky,1,0,2,0.0,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,-0.001,/players/damian-lillard-1.html
183,2010-11,Tennessee,SEC,1,0,3,0.0,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.008,/players/jordan-mcrae-1.html


In [31]:
col_conf_adv = col_conf_adv_raw.fillna(float(0.0))

In [32]:
col_conf_adv.dtypes #OK

SEASON          object
SCHOOL          object
CONF            object
G                int64
GS               int64
MP               int64
PER            float64
TS%            float64
eFG%           float64
3PAr           float64
FTr            float64
PProd            int64
ORB%           float64
DRB%           float64
TRB%           float64
AST%           float64
STL%           float64
BLK%           float64
TOV%           float64
USG%           float64
OWS            float64
DWS            float64
WS             float64
WS/40          float64
COLLEGE_URL     object
dtype: object

In [33]:
col_conf_adv.count()

SEASON         302
SCHOOL         302
CONF           302
G              302
GS             302
MP             302
PER            302
TS%            302
eFG%           302
3PAr           302
FTr            302
PProd          302
ORB%           302
DRB%           302
TRB%           302
AST%           302
STL%           302
BLK%           302
TOV%           302
USG%           302
OWS            302
DWS            302
WS             302
WS/40          302
COLLEGE_URL    302
dtype: int64

In [34]:
col_conf_adv.to_csv('Clean_data/Data_Clean_V1/Stats_college_conf_adv.csv')

### 2.4. col_conf_pg_raw

In [37]:
col_conf_pg_raw['year'] = col_conf_pg_raw['SEASON'].apply(lambda x: int(x[:4]))
col_conf_pg_raw = col_conf_pg_raw[col_conf_pg_raw['year']>1985]
col_conf_pg_raw = col_conf_pg_raw.drop(['year'],1)

In [38]:
col_conf_pg_raw.head()

,SEASON,SCHOOL,CONF,G,GS,MP,FG,FGA,FG%,2P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,COLLEGE_URL,3P%
0,2012-13,Pitt,Big East,17,17,24.3,2.9,5.4,0.543,2.9,...,3.6,6.1,0.6,0.6,2.2,1.1,1.7,6.9,/players/steven-adams-1.html,NaN
1,2016-17,Kentucky,SEC,18,18,30.9,4.8,7.2,0.667,4.8,...,4.8,7.4,0.4,0.8,1.6,1.7,2.7,13.9,/players/edrice-adebayo-1.html,0.000
2,2016-17,Texas,Big 12,18,18,34.6,6.6,10.9,0.599,6.6,...,6.5,9.8,0.7,0.7,1.7,2.8,1.9,16.2,/players/jarrett-allen-1.html,0.000
3,2017-18,Dayton,A-10,16,0,13.3,1.8,2.9,0.609,1.7,...,1.7,2.6,0.4,0.2,0.7,1.1,1.7,5.2,/players/kostas-antetokounmpo-1.html,0.200
4,2015-16,Indiana,Big Ten,18,0,15.6,2.2,4.2,0.533,1.7,...,2.0,3.3,0.6,0.9,0.7,0.6,1.7,5.8,/players/og-anunoby-1.html,0.409


In [39]:
col_conf_pg_raw.isnull().sum()

SEASON          0
SCHOOL          0
CONF            0
G               0
GS              0
MP              0
FG              0
FGA             0
FG%             2
2P              0
2PA             0
2P%             2
3P              0
3PA             0
FT              0
FTA             0
FT%             2
ORB             0
DRB             0
TRB             0
AST             0
STL             0
BLK             0
TOV             0
PF              0
PTS             0
COLLEGE_URL     0
3P%            38
dtype: int64

In [40]:
col_conf_pg_raw[col_conf_pg_raw['2P%'].isna()] #Estos jugadores casi no jugaron en la temporada 2010/11. Lo llenamos con 0.

,SEASON,SCHOOL,CONF,G,GS,MP,FG,FGA,FG%,2P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,COLLEGE_URL,3P%
171,2010-11,Weber State,Big Sky,1,0,2.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/players/damian-lillard-1.html,NaN
183,2010-11,Tennessee,SEC,1,0,3.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/players/jordan-mcrae-1.html,NaN


In [41]:
_3P = col_conf_pg_raw[['COLLEGE_URL', '3P', '3PA', '3P%']] #Estos jugadores no tiraron de 3. Les corresponde un 3P% de 0.
_3P[_3P['3P%'].isna()].head()

,COLLEGE_URL,3P,3PA,3P%
0,/players/steven-adams-1.html,0.0,0.0,NaN
17,/players/jordan-bell-1.html,0.0,0.0,NaN
18,/players/jordan-bell-1.html,0.0,0.0,NaN
40,/players/devontae-cacok-1.html,0.0,0.0,NaN
41,/players/devontae-cacok-1.html,0.0,0.0,NaN


In [42]:
col_conf_pg = col_conf_pg_raw.fillna(float(0.0))

In [43]:
col_conf_pg.dtypes #OK

SEASON          object
SCHOOL          object
CONF            object
G                int64
GS               int64
MP             float64
FG             float64
FGA            float64
FG%            float64
2P             float64
2PA            float64
2P%            float64
3P             float64
3PA            float64
FT             float64
FTA            float64
FT%            float64
ORB            float64
DRB            float64
TRB            float64
AST            float64
STL            float64
BLK            float64
TOV            float64
PF             float64
PTS            float64
COLLEGE_URL     object
3P%            float64
dtype: object

In [44]:
col_conf_pg.count()

SEASON         302
SCHOOL         302
CONF           302
G              302
GS             302
MP             302
FG             302
FGA            302
FG%            302
2P             302
2PA            302
2P%            302
3P             302
3PA            302
FT             302
FTA            302
FT%            302
ORB            302
DRB            302
TRB            302
AST            302
STL            302
BLK            302
TOV            302
PF             302
PTS            302
COLLEGE_URL    302
3P%            302
dtype: int64

In [45]:
col_conf_pg.to_csv('Clean_data/Data_Clean_V1/Stats_College_conf_pg.csv')

### 2.5. col_conf_tot_raw

In [46]:
col_conf_tot_raw['year'] = col_conf_tot_raw['SEASON'].apply(lambda x: int(x[:4]))
col_conf_tot_raw_1 = col_conf_tot_raw[col_conf_tot_raw['year']>1985]

In [47]:
col_conf_tot_raw_1.head()

,SEASON,SCHOOL,CONF,G,GS,MP,FG,FGA,FG%,2P,...,TRB,AST,STL,BLK,TOV,PF,PTS,COLLEGE_URL,3P%,year
0,2012-13,Pitt,Big East,17,17,413,50,92,0.543,50,...,104,10,10,37,19,29,117,/players/steven-adams-1.html,NaN,2012
1,2016-17,Kentucky,SEC,18,18,557,86,129,0.667,86,...,133,8,14,29,31,49,251,/players/edrice-adebayo-1.html,0.000,2016
2,2016-17,Texas,Big 12,18,18,622,118,197,0.599,118,...,176,12,12,31,51,35,291,/players/jarrett-allen-1.html,0.000,2016
3,2017-18,Dayton,A-10,16,0,213,28,46,0.609,27,...,41,6,3,11,18,27,83,/players/kostas-antetokounmpo-1.html,0.200,2017
4,2015-16,Indiana,Big Ten,18,0,281,40,75,0.533,31,...,59,11,17,13,11,30,105,/players/og-anunoby-1.html,0.409,2015


In [48]:
col_conf_tot_raw_1.isnull().sum()

SEASON          0
SCHOOL          0
CONF            0
G               0
GS              0
MP              0
FG              0
FGA             0
FG%             2
2P              0
2PA             0
2P%             2
3P              0
3PA             0
FT              0
FTA             0
FT%             2
ORB             0
DRB             0
TRB             0
AST             0
STL             0
BLK             0
TOV             0
PF              0
PTS             0
COLLEGE_URL     0
3P%            38
year            0
dtype: int64

In [49]:
col_conf_tot_raw_1[col_conf_tot_raw_1['2P%'].isna()]#Estos jugadores casi no jugaron en la temporada 2010/11. Lo llenamos con 0.

,SEASON,SCHOOL,CONF,G,GS,MP,FG,FGA,FG%,2P,...,TRB,AST,STL,BLK,TOV,PF,PTS,COLLEGE_URL,3P%,year
171,2010-11,Weber State,Big Sky,1,0,2,0,0,NaN,0,...,0,0,0,0,0,0,0,/players/damian-lillard-1.html,NaN,2010
183,2010-11,Tennessee,SEC,1,0,3,0,0,NaN,0,...,0,0,0,0,0,0,0,/players/jordan-mcrae-1.html,NaN,2010


In [50]:
_3P_ = col_conf_tot_raw_1[['COLLEGE_URL', '3P', '3PA', '3P%']] #Estos jugadores no tiraron de 3. Les corresponde un 3P% de 0.
_3P_[_3P['3P%'].isna()].head()

,COLLEGE_URL,3P,3PA,3P%
0,/players/steven-adams-1.html,0,0,NaN
17,/players/jordan-bell-1.html,0,0,NaN
18,/players/jordan-bell-1.html,0,0,NaN
40,/players/devontae-cacok-1.html,0,0,NaN
41,/players/devontae-cacok-1.html,0,0,NaN


In [51]:
col_conf_tot = col_conf_tot_raw_1.fillna(float(0.0))

In [52]:
col_conf_tot.dtypes #OK

SEASON          object
SCHOOL          object
CONF            object
G                int64
GS               int64
MP               int64
FG               int64
FGA              int64
FG%            float64
2P               int64
2PA              int64
2P%            float64
3P               int64
3PA              int64
FT               int64
FTA              int64
FT%            float64
ORB              int64
DRB              int64
TRB              int64
AST              int64
STL              int64
BLK              int64
TOV              int64
PF               int64
PTS              int64
COLLEGE_URL     object
3P%            float64
year             int64
dtype: object

In [53]:
col_conf_tot.count()

SEASON         302
SCHOOL         302
CONF           302
G              302
GS             302
MP             302
FG             302
FGA            302
FG%            302
2P             302
2PA            302
2P%            302
3P             302
3PA            302
FT             302
FTA            302
FT%            302
ORB            302
DRB            302
TRB            302
AST            302
STL            302
BLK            302
TOV            302
PF             302
PTS            302
COLLEGE_URL    302
3P%            302
year           302
dtype: int64

In [54]:
col_conf_tot.to_csv('Clean_data/Data_Clean_V1/Stats_College_conf_tot.csv')

### 2.6. col_rs_adv_raw

In [55]:
col_rs_adv_raw['year'] = col_rs_adv_raw['SEASON'].apply(lambda x: int(x[:4]))
col_rs_adv_raw = col_rs_adv_raw[col_rs_adv_raw['year']>1985]
col_rs_adv_raw = col_rs_adv_raw.drop(['year'],1)

In [56]:
col_rs_adv_raw.head()

,SEASON,SCHOOL,CONF,G,MP,TS%,eFG%,3PAr,FTr,TOV%,...,ORB%,DRB%,TRB%,AST%,STL%,BLK%,USG%,OBPM,DBPM,BPM
0,1988-89,LSU,SEC,32.0,1180.0,0.579,0.543,0.292,0.271,11.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1989-90,LSU,SEC,32.0,1202.0,0.584,0.527,0.372,0.317,13.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1995-96,California,Pac-10,28.0,972.0,0.571,0.528,0.053,0.626,14.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2012-13,Pitt,Big East,32.0,749.0,0.555,0.571,0.000,0.400,14.4,...,15.3,21.0,18.0,5.4,1.9,11.2,17.7,4.0,4.2,8.1
8,2016-17,Kentucky,SEC,38.0,1145.0,0.624,0.599,0.000,0.831,13.9,...,11.1,17.3,14.2,4.7,1.2,4.9,18.5,4.5,3.6,8.2


In [57]:
col_rs_adv_raw.dtypes #OK

SEASON          object
SCHOOL          object
CONF            object
G              float64
MP             float64
TS%            float64
eFG%           float64
3PAr           float64
FTr            float64
TOV%           float64
COLLEGE_URL     object
GS             float64
OWS            float64
DWS            float64
WS             float64
WS/40          float64
PER            float64
PProd          float64
ORB%           float64
DRB%           float64
TRB%           float64
AST%           float64
STL%           float64
BLK%           float64
USG%           float64
OBPM           float64
DBPM           float64
BPM            float64
dtype: object

In [58]:
col_rs_adv_raw.isnull().sum()

SEASON            0
SCHOOL            0
CONF              0
G                 1
MP                2
TS%               1
eFG%              1
3PAr              1
FTr               1
TOV%             63
COLLEGE_URL       0
GS              182
OWS             537
DWS             537
WS              537
WS/40           537
PER            1166
PProd          1150
ORB%           1150
DRB%           1150
TRB%            887
AST%            886
STL%           1147
BLK%            886
USG%            886
OBPM           1192
DBPM           1192
BPM            1192
dtype: int64

In [59]:
col_rs_adv_raw.count()

SEASON         1503
SCHOOL         1503
CONF           1503
G              1502
MP             1501
TS%            1502
eFG%           1502
3PAr           1502
FTr            1502
TOV%           1440
COLLEGE_URL    1503
GS             1321
OWS             966
DWS             966
WS              966
WS/40           966
PER             337
PProd           353
ORB%            353
DRB%            353
TRB%            616
AST%            617
STL%            356
BLK%            617
USG%            617
OBPM            311
DBPM            311
BPM             311
dtype: int64

In [60]:
#profile_rs_adv = col_rs_adv_raw.profile_report()
#profile_rs_adv.to_file("Reports/Raw_data/report_rs_adv_raw.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [61]:
col_rs_adv = col_rs_adv_raw.fillna(float(0.0))

In [64]:
col_rs_adv.to_csv('Clean_data/Data_Clean_V1/Stats_college_rs_adv.csv')

### 2.7. col_rs_pg_raw

In [65]:
col_rs_pg_raw['year'] = col_rs_pg_raw['SEASON'].apply(lambda x: int(x[:4]))
col_rs_pg_raw = col_rs_pg_raw[col_rs_pg_raw['year']>1985]
col_rs_pg_raw = col_rs_pg_raw.drop(['year'],1)

In [66]:
col_rs_pg_raw.head()

,SEASON,SCHOOL,CONF,G,MP,FG,FGA,FG%,2P,2PA,...,TOV,PF,PTS,SOS,COLLEGE_URL,GS,ORB,DRB,Unnamed: 27,Unnamed: 25
0,1988-89,LSU,SEC,32,36.9,11.2,23.1,0.486,8.6,16.3,...,3.5,2.2,30.2,6.44,/players/mahmoud-abdul-rauf-1.html,NaN,NaN,NaN,NaN,NaN
1,1989-90,LSU,SEC,32,37.6,9.5,20.7,0.461,6.8,13.0,...,3.7,2.1,27.8,7.64,/players/mahmoud-abdul-rauf-1.html,NaN,NaN,NaN,NaN,NaN
2,1995-96,California,Pac-10,28,34.7,7.4,14.2,0.518,7.1,13.5,...,3.1,2.1,21.1,6.44,/players/shareef-abdur-rahim-1.html,28.0,NaN,NaN,NaN,NaN
7,2012-13,Pitt,Big East,32,23.4,3.1,5.5,0.571,3.1,5.5,...,1.1,1.6,7.2,5.66,/players/steven-adams-1.html,32.0,2.8,3.5,NaN,NaN
8,2016-17,Kentucky,SEC,38,30.1,4.5,7.5,0.599,4.5,7.5,...,1.7,2.6,13.0,10.49,/players/edrice-adebayo-1.html,38.0,3.1,4.9,NaN,NaN


In [67]:
col_rs_pg_raw.dtypes #todo object!!!!

SEASON         object
SCHOOL         object
CONF           object
G              object
MP             object
FG             object
FGA            object
FG%            object
2P             object
2PA            object
2P%            object
3P             object
3PA            object
3P%            object
FT             object
FTA            object
FT%            object
TRB            object
AST            object
STL            object
BLK            object
TOV            object
PF             object
PTS            object
SOS            object
COLLEGE_URL    object
GS             object
ORB            object
DRB            object
Unnamed: 27    object
Unnamed: 25    object
dtype: object

In [68]:
col_rs_pg_raw.isnull().sum()

SEASON            0
SCHOOL           14
CONF             14
G                 1
MP                2
FG                1
FGA               1
FG%               1
2P                1
2PA               1
2P%               1
3P                1
3PA               1
3P%             186
FT                1
FTA               1
FT%               2
TRB               1
AST               1
STL               1
BLK               1
TOV              17
PF               18
PTS               1
SOS               0
COLLEGE_URL       0
GS              182
ORB             767
DRB             767
Unnamed: 27    1503
Unnamed: 25    1526
dtype: int64

In [69]:
col_rs_pg_raw.count()

SEASON         1526
SCHOOL         1512
CONF           1512
G              1525
MP             1524
FG             1525
FGA            1525
FG%            1525
2P             1525
2PA            1525
2P%            1525
3P             1525
3PA            1525
3P%            1340
FT             1525
FTA            1525
FT%            1524
TRB            1525
AST            1525
STL            1525
BLK            1525
TOV            1509
PF             1508
PTS            1525
SOS            1526
COLLEGE_URL    1526
GS             1344
ORB             759
DRB             759
Unnamed: 27      23
Unnamed: 25       0
dtype: int64

In [70]:
#profile_rs_pg = col_rs_pg_raw.profile_report()
#profile_rs_pg.to_file("Reports/Raw_data/report_rs_pg_raw.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [71]:
col_rs_pg = col_rs_pg_raw.fillna(float(0.0))

In [73]:
col_rs_pg.to_csv('Clean_data/Data_Clean_V1/Stats_college_rs_pg.csv')

### 2.8. col_rs_tot_raw

In [74]:
col_rs_tot_raw['year'] = col_rs_tot_raw['SEASON'].apply(lambda x: int(x[:4]))
col_rs_tot_raw = col_rs_tot_raw[col_rs_tot_raw['year']>1985]
col_rs_tot_raw = col_rs_tot_raw.drop(['year'],1)

In [75]:
col_rs_tot_raw.head()

,SEASON,SCHOOL,CONF,G,MP,FG,FGA,FG%,2P,2PA,...,AST,STL,BLK,TOV,PF,PTS,COLLEGE_URL,GS,ORB,DRB
0,1988-89,LSU,SEC,32.0,1180.0,359.0,739.0,0.486,275.0,523.0,...,130.0,55.0,6.0,113.0,70.0,965.0,/players/mahmoud-abdul-rauf-1.html,NaN,NaN,NaN
1,1989-90,LSU,SEC,32.0,1202.0,305.0,662.0,0.461,217.0,416.0,...,102.0,52.0,1.0,117.0,66.0,889.0,/players/mahmoud-abdul-rauf-1.html,NaN,NaN,NaN
2,1995-96,California,Pac-10,28.0,972.0,206.0,398.0,0.518,198.0,377.0,...,29.0,52.0,35.0,87.0,58.0,590.0,/players/shareef-abdur-rahim-1.html,28.0,NaN,NaN
7,2012-13,Pitt,Big East,32.0,749.0,100.0,175.0,0.571,100.0,175.0,...,20.0,21.0,65.0,35.0,52.0,231.0,/players/steven-adams-1.html,32.0,90.0,113.0
8,2016-17,Kentucky,SEC,38.0,1145.0,170.0,284.0,0.599,170.0,284.0,...,32.0,26.0,57.0,64.0,99.0,494.0,/players/edrice-adebayo-1.html,38.0,118.0,186.0


In [76]:
col_rs_tot_raw.dtypes #OK

SEASON          object
SCHOOL          object
CONF            object
G              float64
MP             float64
FG             float64
FGA            float64
FG%            float64
2P             float64
2PA            float64
2P%            float64
3P             float64
3PA            float64
3P%            float64
FT             float64
FTA            float64
FT%            float64
TRB            float64
AST            float64
STL            float64
BLK            float64
TOV            float64
PF             float64
PTS            float64
COLLEGE_URL     object
GS             float64
ORB            float64
DRB            float64
dtype: object

In [77]:
col_rs_tot_raw.isnull().sum()

SEASON           0
SCHOOL           0
CONF             0
G                1
MP               2
FG               1
FGA              1
FG%              1
2P               1
2PA              1
2P%              1
3P               1
3PA              1
3P%            186
FT               1
FTA              1
FT%              2
TRB              1
AST              1
STL              1
BLK              1
TOV             17
PF              18
PTS              1
COLLEGE_URL      0
GS             182
ORB            767
DRB            767
dtype: int64

In [78]:
col_rs_tot_raw.count()

SEASON         1503
SCHOOL         1503
CONF           1503
G              1502
MP             1501
FG             1502
FGA            1502
FG%            1502
2P             1502
2PA            1502
2P%            1502
3P             1502
3PA            1502
3P%            1317
FT             1502
FTA            1502
FT%            1501
TRB            1502
AST            1502
STL            1502
BLK            1502
TOV            1486
PF             1485
PTS            1502
COLLEGE_URL    1503
GS             1321
ORB             736
DRB             736
dtype: int64

In [79]:
col_rs_tot = col_rs_tot_raw.fillna(float(0.0))

In [81]:
col_rs_tot.to_csv('Clean_data/Data_Clean_V1/Stats_College_rs_tot.csv')

### 2.9. col_extra_info_raw

In [82]:
pd.set_option('max_columns', None)
col_extra_info_raw.head()

,HS_PARADE_AA,CONSENSUS_AA,SEC_POY,ALL-SEC,SEC_ALL-FRESHMAN,COLLEGE_URL,PAC-10_POY,ALL-PAC-12,PAC-12_ALL-FRESHMAN,PAC-10_ROY,ALL-BIG_EAST,BIG_EAST_ALL-FRESHMAN,2017_ALL-SEC_TOURNEY,NCAA_ALL-REGION,ALL-BIG_12,2006_ALL-BIG_12_TOURNEY,BIG_12_ALL-DEFENSE,BIG_12_DPOY,WAC_POY,ACC_ALL-FRESHMAN,BIG_12_ALL-FRESHMAN,BIG_EAST_POY,ALL-BIG_EAST_TOURNEY,BIG_12_POY,2004_ALL-BIG_12_TOURNEY,BIG_12_TOURNEY_MVP,NCAA_CHAMPION,ALL-ACC,1990_ALL-ACC_TOURNEY,ACC_ROY,2007_ALL-PAC-12_TOURNEY,NCAA_ALL-TOURNEY,NCAA_TOURNEY_MOP,2003_ALL-BIG_EAST_TOURNEY,BIG_EAST_ROY,BIG_WEST_POY,2008_ALL-BIG_12_TOURNEY,2009_ALL-PAC-12_TOURNEY,PAC-12_POY,2018_ALL-PAC-12_TOURNEY,PAC-12_ALL-DEFENSE,PAC-12_ROY,PAC-12_TOURNEY_MVP,ACC_POY,2018_ALL-ACC_TOURNEY,NAC_POY,CAA_POY,ALL-ACC_TOURNEY,2021_ALL-AAC_TOURNEY,ACC_6MOY,1989_ALL-BIG_EAST_TOURNEY,AP_POY,WOODEN_AWARD,NAISMITH_AWARD,RUPP_TROPHY,ACC_ALL-DEFENSE,ACC_TOURNEY_MVP,2012_ALL-SEC_TOURNEY,BIG_12_ROY,PAC-12_DPOY,SEC_ROY,2010_ALL-SEC_TOURNEY,MWC_POY,SEC_6MOY,1997_ALL-ACC_TOURNEY,2016_ALL-PAC-12_TOURNEY,1984_ALL-ACC_TOURNEY,1999_ALL-ACC_TOURNEY,ALL-SEC_TOURNEY,SEC_DPOY,2018_ALL-BIG_EAST_TOURNEY,BIG_EAST_DPOY,BIG_EAST_TOURNEY_MVP,ACC_DPOY,1987_ALL-ACC_TOURNEY,BIG_TEN_ALL-DEFENSE,BIG_TEN_ALL-FRESHMAN,ALL-PAC-12_TOURNEY,BIG_TEN_POY,ALL-BIG_TEN,BIG_TEN_ROY,2002_ALL-BIG_EAST_TOURNEY,ALL-CAA,CAA_ALL-DEFENSE,A-10_POY,1998_ALL-ACC_TOURNEY,BIG_EAST_MIP,SEC_ALL-DEFENSE,2015_ALL-SEC_TOURNEY,SEC_TOURNEY_MVP,ALL-BIG_12_TOURNEY,1988_ALL-SEC_TOURNEY,1991_ALL-ACC_TOURNEY,2004_ALL-PAC-12_TOURNEY,WCC_POY,OVC_POY,ALL-WCC,2019_ALL-WCC_TOURNEY,WCC_DPOY,WCC_ROY,AEC_POY,HORIZON_POY,1990_ALL-BIG_EAST_TOURNEY,ACC_MIP,2008_ALL-PAC-12_TOURNEY,2007_ALL-BIG_TEN_TOURNEY,2013_ALL-BIG_EAST_TOURNEY,2015_ALL-ACC_TOURNEY,2021_ALL-BIG_12_TOURNEY,METRO_POY,SOUTHERN_POY,MAC_POY,1985_ALL-ACC_TOURNEY,2006_ALL-SEC_TOURNEY,2004_ALL-ACC_TOURNEY,BIG_EAST_6MOY,2012_ALL-BIG_EAST_TOURNEY,2021_ALL-BIG_TEN_TOURNEY,BIG_TEN_TOURNEY_MVP,2006_ALL-ACC_TOURNEY,SOUTHLAND_POY,2007_ALL-BIG_12_TOURNEY,2006_ALL-PAC-12_TOURNEY,2010_ALL-ACC_TOURNEY,SUN_BELT_POY,2000_ALL-BIG_12_TOURNEY,2009_ALL-BIG_EAST_TOURNEY,2002_ALL-BIG_12_TOURNEY,2005_ALL-PAC-12_TOURNEY,PAC-10_DPOY,2018_ALL-SEC_TOURNEY,PCAA_POY,MW_COLL_POY,2012_ALL-BIG_TEN_TOURNEY,BIG_TEN_6MOY,2007_ALL-BIG_EAST_TOURNEY,2001_ALL-BIG_EAST_TOURNEY,GMWC_POY,ALL-PAC-10,2009_ALL-PAC-10_TOURNEY,PAC-10_ALL-FRESHMAN,1996_ALL-ACC_TOURNEY,2014_ALL-AAC_TOURNEY,AAC_MIP,2004_ALL-BIG_TEN_TOURNEY,MVC_POY,2001_ALL-ACC_TOURNEY,2005_ALL-BIG_TEN_TOURNEY,2007_ALL-SEC_TOURNEY,1991_ALL-SEC_TOURNEY,2003_ALL-ACC_TOURNEY,SWAC_POY,ALL-A-10,2021_ALL-A-10_TOURNEY,A-10_ALL-FRESHMAN,BIG_TEN_DPOY,1986_ALL-BIG_EAST_TOURNEY,2021_ALL-SEC_TOURNEY,1988_ALL-PAC-12_TOURNEY,BIG_8_POY,2011_ALL-SEC_TOURNEY,1996_ALL-BIG_EAST_TOURNEY,2005_ALL-SEC_TOURNEY,ECACN_POY,BIG_SKY_POY,1998_ALL-BIG_12_TOURNEY,IVY_POY,2017_ALL-PAC-12_TOURNEY,1994_ALL-BIG_EAST_TOURNEY,CUSA_POY,1989_ALL-ACC_TOURNEY,SWC_POY,PATRIOT_POY,1995_ALL-SEC_TOURNEY,1997_ALL-SEC_TOURNEY,ALL-MWC,ALL-MWC_TOURNEY,MWC_TOURNEY_MVP,1998_ALL-BIG_TEN_TOURNEY,1985_NIT_MVP,1987_ALL-PAC-12_TOURNEY,2021_ALL-PAC-12_TOURNEY,1998_ALL-SEC_TOURNEY,2010_ALL-BIG_EAST_TOURNEY,ALL-OVC,2019_ALL-OVC_TOURNEY,OVC_TOURNEY_MVP,2011_ALL-BIG_12_TOURNEY,2016_ALL-SEC_TOURNEY,1991_ALL-BIG_EAST_TOURNEY,2019_ALL-BIG_EAST_TOURNEY,2000_ALL-BIG_TEN_TOURNEY,2009_ALL-BIG_12_TOURNEY,2001_ALL-SEC_TOURNEY,2014_ALL-SEC_TOURNEY,1999_ALL-BIG_TEN_TOURNEY,2012_ALL-PAC-12_TOURNEY,ECACS_POY,TAAC_POY,1992_ALL-ACC_TOURNEY,2018_ALL-AAC_TOURNEY,MAAC_POY,2013_ALL-BIG_12_TOURNEY,1995_ALL-ACC_TOURNEY,ECACM_POY,MEAC_POY,2017_ALL-ACC_TOURNEY,1997_ALL-BIG_EAST_TOURNEY,2011_ALL-PAC-12_TOURNEY,2007_ALL-ACC_TOURNEY,2019_ALL-PAC-12_TOURNEY,1997_ALL-BIG_12_TOURNEY,PAC-10_MIP,2011_ALL-BIG_EAST_TOURNEY,2002_ALL-PAC-12_TOURNEY,2019_ALL-ACC_TOURNEY,2014_ALL-BIG_12_TOURNEY,1989_NIT_MVP,1988_ALL-ACC_TOURNEY,1998_ALL-BIG_EAST_TOURNEY,1993_ALL-ACC_TOURNEY,2013_ALL-BIG_TEN_TOURNEY
0,1.0,2.0,2,2,1.0,/players/mahmoud-abdul-rauf-1.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

Hay demasiada información con demasiados datos vacíos. Vamos a quedarnos solo con los campeones de la NCAA (NCAA_CHAMPION) y mejores jugadores de cada año (NCAA_ALL-TOURNEY)

In [83]:
col = ['NCAA_CHAMPION', 'NCAA_ALL-TOURNEY', 'COLLEGE_URL', 'NCAA_TOURNEY_MOP']
col_extra_info_1 = col_extra_info_raw[col]

In [84]:
col_extra_info_1.dtypes #OK

NCAA_CHAMPION       float64
NCAA_ALL-TOURNEY    float64
COLLEGE_URL          object
NCAA_TOURNEY_MOP    float64
dtype: object

In [85]:
col_extra_info_1.count()

NCAA_CHAMPION        63
NCAA_ALL-TOURNEY     67
COLLEGE_URL         577
NCAA_TOURNEY_MOP     19
dtype: int64

In [57]:
63/12

5.25

In [86]:
col_extra_info_1.to_csv('Clean_data/Data_Clean_V1/College_extra_info.csv')